In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
np.random.seed(123549)
df_1 = pd.DataFrame(np.random.poisson(50,60), index=pd.DatetimeIndex(start='2011-01-01', periods=60, freq='D', name='Observation Date'), columns=['No. Events'])

C:\Users\littlejack\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  


# Assignment 1
The records in `df_1` has erroneously been set to be recorded from Jan 1, 2011. The records were in fact from May 2, 2011. Adjust the index accordingly.

<details><summary>Hint 1</summary>
    <p>
        A pandas DataFrame as a <b>tshift</b> method, that allows you to shift the DatetimeIndex by a given number of steps at the DatetimeIndex's frequency
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Pandas assumes that individual indices are independent. It will therefore shift every index seperately. To keep the proper relation between dates, calculate the offset for the first date in days and offset all indices by that number of days
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        Pandas has a generic <b>DateOffset</b> object in <b>pd.tseries.offsets.DateOffset</b>, that allows you to specify the number of years, months and days to offset a date by
    </p>
</details>
<details><summary>Hint 4</summary>
    <p>
        Subtracting two pandas dates returns a <b>timedelta</b>. A <b>timedelta</b> has an attribute <b>days</b>, that returns the number of days between the two dates as an integer
    </p>
</details>

In [15]:
df1_date=df_1.set_index(pd.Index(pd.date_range(start='2011/05/02',periods=60)))
df1_date.index.name = 'Observation Date'
df1_date
                  

,No. Events
Observation Date,
2011-05-02,43
2011-05-03,48
2011-05-04,43
2011-05-05,55
2011-05-06,48
2011-05-07,53
2011-05-08,66
2011-05-09,52
2011-05-10,59


# Assignment 2
Every Thursday the business tallies the total number of events in the previous week (Thursday to Wednesday). Compute the total number of event in every Thursday to Wednesday period and return a pandas Series where the index is the date of the Thursday the week's data is being tallied.
<details><summary>Hint 1</summary>
    <p>
        The pandas offset object <b>pd.tseries.offsets.Week</b> has a parameter <b>weekday</b>, that allows you to specify which day of the week a date should be offset to. Note: When specifying e.g. Monday as the weekday, every Monday will be offset by a full week as pandas offsets to the following weekday. The parameter weekday is defined by an integer, where Monday is 0 and Sunday is 6
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Pandas groupby can be passed a Series not in the DataFrame, e.g. an offset index, and it will then group by the values in that Series
    </p>
</details>

In [29]:
period = df1_date.index.to_period("W-WED")
answer2 = df1_date.groupby(period).sum()
answer2.index=answer2.index.to_series().astype(str)
answer2.index=answer2.index.map(lambda x: x.split('/')[0])
answer2

,No. Events
Observation Date,
2011-04-28,134
2011-05-05,375
2011-05-12,331
2011-05-19,353
2011-05-26,329
2011-06-02,326
2011-06-09,354
2011-06-16,302
2011-06-23,338


# Assignment 3
For each calendar month find the date where the most events occurred
<details><summary>Hint 1</summary>
    <p>
        Pandas as an offset object <b>MonthBegin</b>, that offsets a date to the beginning over the month
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Pandas groupby can be passed a Series not in the DataFrame, e.g. an offset index, and it will then group by the values in that Series
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        A pandas DataFrame has a method <b>nlargest</b> which can return the rows of a columns, that contains the largest values
    </p>
</details>

In [53]:
answer3 = df1_date.copy()
answer3.groupby(df1_date.index.to_period("M")).apply(lambda x: x.nlargest(n=1,columns=["No. Events"]))


,,No. Events
Observation Date,Observation Date,
2011-05,2011-05-08,66
2011-06,2011-06-05,62


# Assignment 4
The event counts in `df_1` signals the number of contacts the business gets in a day. To plan the weekend staffing, the capacity is planned as the average of the week (Monday to Friday) plus one standard deviation for the same period. How many of the weekend days did the number of contacts exceed capacity?
<details><summary>Hint 1</summary>
    <p>
        A DatetimeIndex has the attributes <b>week</b> and <b>dayofweek</b>, that returns the week number and the day of week number, respectively, for each date.
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Pandas groupby can be passed a Series not in the DataFrame, e.g. an offset index, and it will then group by the values in that Series
    </p>
</details>

# Assignment 5
Find the daily average of No. Events up until, but not including, the first day that No. Events exceeds 60 for each month.
<details><summary>Hint 1</summary>
    <p>
        Applying the <b>cummax</b> method to a Boolean pandas Series will return 0 up until the first True value, and the return 1 from there on
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        A DatetimeIndex has the attribute <b>month</b>, that returns the month number of each date
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        Pandas groupby can be passed a Series not in the DataFrame, e.g. an offset index, and it will then group by the values in that Series
    </p>
</details>

In [54]:
answer5= df1_date

In [55]:
answer5['total'] = answer5.groupby(answer5.index.to_period("M")).cummax()
answer5

,No. Events,total
Observation Date,,
2011-05-02,43,43
2011-05-03,48,48
2011-05-04,43,48
2011-05-05,55,55
2011-05-06,48,55
2011-05-07,53,55
2011-05-08,66,66
2011-05-09,52,66
2011-05-10,59,66


In [61]:
answer5 = answer5.loc[answer5['No. Events'] > 60]
answer5

,No. Events,total
Observation Date,,
2011-05-08,66,66
2011-06-05,62,62
2011-06-14,61,62


In [62]:
answer5["No. Events"].groupby(answer5.index.to_period("M")).mean()

Observation Date
2011-05    66.0
2011-06    61.5
Freq: M, Name: No. Events, dtype: float64